## Pytesseract

In [1]:
import pytesseract
from PIL import Image

In [4]:
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

text = pytesseract.image_to_string(Image.open("../data/mail2.png"))
print(text)

M Gmail

@ Compose
Inbox

yy Starred
Snoozed
& Sent

D Drafts

vy More
Labels

® Form Responses
Hostel

® Lost and Found

® Not Required

© Placement Cell IIIT...

Q Search mail

os

@rctver @ Ev.)
6 0° @ 4 © G ® 3D: 30f6759 << > Ey
Appointment Confirmation with Dr. Meera Joshi_ !nbox @ @
©S22B1027 MADUPU SAI PRANAV <cs221027@iitdm.ac.in> 4:45PM (6 minutes ago) yy ¢
tome ~

Dear Ms. Priya Rathi,

We are pleased to confirm your consultation with Dr. Meera Joshi (Dermatologist) scheduled for tomorrow afternoon at Fortis Hospital, New
Delhi.

Please ensure you carry any previous prescriptions or test results during your visit. If you are unable to attend, kindly inform us at least 24 hours
in advance.

Warm regards,

Fortis Hospital — Patient Care Desk

fi. 7 \ fe op
(© Reply )( @ Forward )
NX JN 7

@e a



In [5]:
import re

# Keep only letters, numbers, and basic punctuation
clean_text = re.sub(r'[^A-Za-z0-9,.!?@#\s]', '', text)

# Remove extra whitespaces or blank lines
clean_text = "\n".join([line.strip() for line in clean_text.splitlines() if line.strip()])

print(clean_text)

M Gmail
@ Compose
Inbox
yy Starred
Snoozed
Sent
D Drafts
vy More
Labels
Form Responses
Hostel
Lost and Found
Not Required
Placement Cell IIIT...
Q Search mail
os
@rctver @ Ev.
6 0 @ 4  G  3D 30f6759   Ey
Appointment Confirmation with Dr. Meera Joshi !nbox @ @
S22B1027 MADUPU SAI PRANAV cs221027@iitdm.ac.in 445PM 6 minutes ago yy
tome
Dear Ms. Priya Rathi,
We are pleased to confirm your consultation with Dr. Meera Joshi Dermatologist scheduled for tomorrow afternoon at Fortis Hospital, New
Delhi.
Please ensure you carry any previous prescriptions or test results during your visit. If you are unable to attend, kindly inform us at least 24 hours
in advance.
Warm regards,
Fortis Hospital  Patient Care Desk
fi. 7  fe op
Reply  @ Forward
NX JN 7
@e a


In [7]:
text = pytesseract.image_to_string(Image.open("../data/note1.png"))
print(text)

Dr. Joshi (Cardiologist) -
Appointment @ 1:30 PM next
saturday



In [10]:
text = pytesseract.image_to_string(Image.open("../data/written_note2.jpeg"))
print(text)

Dentist Bppet

‘Temet




## TrOCR - HuggingFace

In [4]:
# ! pip install transformers
# ! pip install -q torch torchvision --index-url https://download.pytorch.org/whl/cu128
# ! pip install sentencepiece

In [4]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import requests
import torch

# Load processor (handles image → tensor preprocessing)
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-base-handwritten', use_fast=False)
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-base-handwritten')


c:\Users\msaip\Projects\appointment_scheduler\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\msaip\.cache\huggingface\hub\models--microsoft--trocr-base-handwritten. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Fal

In [17]:
from PIL import Image, ImageEnhance, ImageOps

image = Image.open("../data/written_note3.jpeg").convert("RGB")
image = ImageOps.exif_transpose(image) 

# Enhance contrast
gray = image.convert("L")
enhancer = ImageEnhance.Contrast(gray)
image = enhancer.enhance(2.0).convert("RGB")

# Prepare inputs
pixel_values = processor(images=image, return_tensors="pt").pixel_values

# Generate text with beam search
generated_ids = model.generate(pixel_values, num_beams=4, max_length=128, early_stopping=True)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

print("Extracted Text:", generated_text)


Extracted Text: 2 Dentist appointment at 11 30mm. Today


## Mistral AI OCR

In [19]:
# ! pip install mistralai

In [20]:
from mistralai import Mistral

api_key = "Wcn5vM6dh1F7gaKl4ExeBqmfTKTotDBB"
client = Mistral(api_key=api_key)

In [21]:
import base64, mimetypes

def load_image(image_path):
    mime_type, _ = mimetypes.guess_type(image_path)
    with open(image_path, "rb") as image_file:
        image_data = image_file.read()
    base64_encoded = base64.b64encode(image_data).decode('utf-8')
    data_url = f"data:{mime_type};base64,{base64_encoded}"
    return data_url

In [35]:
ocr_response = client.ocr.process(
    model="mistral-ocr-latest",
    document={
        "type": "image_url",
        "image_url": load_image("../data/mail2.png")
    }
)

In [32]:
ocr_response

OCRResponse(pages=[OCRPageObject(index=0, markdown='# Dentist Appointment\n\nTomorrow at 11:30 AM.', images=[], dimensions=OCRPageDimensions(dpi=200, height=1600, width=1200))], model='mistral-ocr-2505-completion', usage_info=OCRUsageInfo(pages_processed=1, doc_size_bytes=39016), document_annotation=None)

In [36]:
for page in ocr_response.pages:
    print(page.markdown)

|  ☐ |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |   |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
|  Gmail |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |   |
|  📸 |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |   |
|  Compose |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |   |
|  Mail |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |   |
|  📸 |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |